<a href="https://colab.research.google.com/github/metatatt/ai2main/blob/main/OpenFDA_Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Calling with Structured Output

### Install Required Libraries

In [ ]:
!pip install -q openai
!pip install newsapi-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


### Upgrade Existing Installation

In [ ]:
!pip install openai -U

### Load Environment Variables

In [ ]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['NEWSAPI_KEY'] = userdata.get('NEWSAPI_KEY')

# News Articles Summarization

### Import Required Libraries

In [ ]:
!pip install newspaper3k

In [ ]:
import openai
from openai import OpenAI
from pydantic import BaseModel, Field
import json
from newspaper import Article # import Article from newspaper3k
from newsapi import NewsApiClient

### Create and Initialize Client

In [ ]:
newsapi = NewsApiClient(api_key=os.environ.get('NEWSAPI_KEY'))
client = OpenAI()

### Create Get News Without Structure

In [ ]:
def get_news_summary_without_structure(query: str) -> str:
    """
    This function searches for a news article based on the query using News API,
    extracts the article content using the newspaper library, and summarizes it using OpenAI's GPT model
    *without* enforcing strict adherence to a structured output schema.
    """

    top_headlines = newsapi.get_everything(q=query,
                                          from_param='2024-08-10',
                                          to='2024-09-10',
                                          language='en',
                                          sort_by='relevancy',
                                          page=2)

    if top_headlines['status'] == 'ok' and top_headlines['articles']:
        article_url = top_headlines['articles'][0]['url']
        article = Article(article_url)
        article.download()
        article.parse()

        summary_prompt = f"Summarize the following news article in 2-3 sentences:\n\n{article.text}"
        summary_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant that summarizes news articles."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content

        return summary

    else:
        print(top_headlines['status'])
        print(top_headlines['articles'])
        raise Exception("No news articles found for the given query.")

### Testing the function

In [ ]:
get_news_summary_without_structure("US Congress")

'The article discusses how the Trump-Vance and Harris-Walz presidential campaigns are approaching blue-collar workers and pro-labor groups differently. Trump is targeting the Teamsters union but has faced criticism for his labor policies, while Harris has garnered support from several pro-labor organizations. Both campaigns are emphasizing their stance on labor issues to appeal to union voters, with Harris-Walz being seen as more pro-labor compared to Trump-Vance. Union membership remains significant in the presidential race, with both campaigns vying for union support.'

### Define Pydantic Models

In [ ]:
class NewsSummaryRequest(BaseModel):
    query: str = Field(..., description="The search query for news articles.")

class NewsSummaryResponse(BaseModel):
    summary: str = Field(..., description="A concise summary of the news article.")
    url: str = Field(..., description="The URL of the news article.")

### Fetch and Summarize News Articles Function with Structure Outputs

# Flow #1

In [ ]:
def get_news_summary(query: str) -> NewsSummaryResponse:
    """
    This function searches for a news article based on the query using News API,
    extracts the article content using the bbc-news library, and summarizes it using OpenAI's GPT model.
    """

    top_headlines = newsapi.get_everything(q='large language models',
                                      from_param='2024-08-10',
                                      to='2024-09-10',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

    if top_headlines['status'] == 'ok' and top_headlines['articles']:
        article_url = top_headlines['articles'][0]['url']
        article = Article(article_url)
        article.download()
        article.parse()

        summary_prompt = f"Summarize the following news article in 2-3 sentences:\n\n{article.text}"
        summary_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant that summarizes news articles."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content

        return NewsSummaryResponse(summary=summary, url=article_url)
    else:
        print(top_headlines['status'])
        print(top_headlines['articles'])
        raise Exception("No news articles found for the given query.")

In [ ]:
test_run = get_news_summary("US Congress news")
print(f"Summary: {test_run.summary}\nURL: {test_run.url}")

ArticleException: Article `download()` failed with ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')) on URL https://www.androidpolice.com/google-pixel-9-changes-from-pixel-8/

In [ ]:
## Wrap the get_news_summary function in Pydantic Model
class NewsSummaryTool(BaseModel):
    def __call__(self, query: str) -> NewsSummaryResponse:
        return get_news_summary(query)

In [ ]:
## create tool with wrapper model
tools = [openai.pydantic_function_tool(
    NewsSummaryTool, name="get_news_summary", description="Get a summary of a news article based on a query")
        ]

### Setup the messages array

In [ ]:
messages = [
    {
        "role": "system",
        "content": """
        You are a helpful assistant that can summarize news articles. Use the supplied tools to assist the user.
        When using a tool, provide all the necessary arguments in a JSON format within the 'arguments' field of the tool call.
        For example, to use the 'get_news_summary' tool,
        provide the 'query' argument like this: {'query': 'latest news about Artificial Intelligence'}.
        """
    },
    {
        "role": "user",
        "content": "Summarize the latest news on US Congress"
    }
]

### Make the initial API call

In [ ]:
response = client.chat.completions.create(
    model='gpt-4',
    messages=messages,
    tools=tools,
    tool_choice={
        "type": "function",
        "function": {
            "name": "get_news_summary"
        },
        "strict": True
    }
)

In [ ]:
result = response.choices[0].message.content
print(result)

None


### Test the applicaiton (Check if the model wants to use a tool)

In [ ]:
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    try:
        news_summary = get_news_summary(arguments['query'])

        function_call_result_message = {
            "role": "tool",
            "name": function_name,
            "content": news_summary.model_dump_json(),
            "tool_call_id": tool_call.id
        }

        messages.append(response.choices[0].message)
        messages.append(function_call_result_message)

        final_response = client.chat.completions.create(
            model='gpt-4o-2024-08-06',
            messages=messages
        )

        print(final_response.choices[0].message.content)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
else:
    print(response.choices[0].message.content)

An error occurred: Article `download()` failed with ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')) on URL https://www.androidpolice.com/google-pixel-9-changes-from-pixel-8/
